In [1]:
!pwd

/home/giakhang/dev/pose_sandbox


In [2]:
import os
import sys
import glob

sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D")

from dataloader_ann import HandArmLandmarksDataset_ANN

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime

import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from csv_writer import fusion_csv_columns_name
import matplotlib.pyplot as plt

import joblib
import sklearn

import json

import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
landmarks_file = "/home/giakhang/dev/pose_sandbox/data/2024-10-29/2024-10-29-14:56/landmarks_all_2024-10-29-14:56.csv"
data = pd.read_csv(landmarks_file)

In [4]:
gt_data = data.loc[:, "left_shoulder_output_x":"right_pinky_tip_output_z"].values  # Columns 324 to the end are outputs (144 features)

In [5]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder",
    "right hip", "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", 
    "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP",
    "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP",
    "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP",
    "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "right elbow",
    "RIGHT_WRIST", "RIGHT_THUMB_CMC", "RIGHT_THUMB_MCP", "RIGHT_THUMB_IP", "RIGHT_THUMB_TIP",
    "RIGHT_INDEX_FINGER_MCP", "RIGHT_INDEX_FINGER_PIP", "RIGHT_INDEX_FINGER_DIP",
    "RIGHT_INDEX_FINGER_TIP", "RIGHT_MIDDLE_FINGER_MCP", "RIGHT_MIDDLE_FINGER_PIP",
    "RIGHT_MIDDLE_FINGER_DIP", "RIGHT_MIDDLE_FINGER_TIP", "RIGHT_RING_FINGER_MCP",
    "RIGHT_RING_FINGER_PIP", "RIGHT_RING_FINGER_DIP", "RIGHT_RING_FINGER_TIP",
    "RIGHT_PINKY_MCP", "RIGHT_PINKY_PIP", "RIGHT_PINKY_DIP", "RIGHT_PINKY_TIP"]

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]

train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200

In [6]:
gt_data.shape

(143, 144)

In [7]:
left_arm_lines = [[0, 1], [1, 5], [5, 10], [5, 14], [10, 14]]
right_arm_lines = [[3, 26], [26, 27], [27, 32], [27, 36], [32, 36]] 

left_hand_lines = [[5, 6], [14, 18], [18, 22], [5, 22],  
                    [6, 7], [7, 8], [8, 9],  # thumb lines
                    [10, 11], [11, 12], [12, 13],  # index lines
                    [14, 15], [15, 16], [16, 17],  # middle lines
                    [18, 19], [19, 20], [20, 21],  # ring lines
                    [22, 23], [23, 24], [24, 25]]  # pinky lines

In [8]:
LANDMARKS_TO_FUSED_IDX = []

LANDMARKS_TO_FUSED = ["left shoulder", "left hip", "right shoulder", "right hip"]
RIGHT_ARM_LANDMARKS_TO_FUSED = ["right elbow", "RIGHT_WRIST", "RIGHT_INDEX_FINGER_MCP", "RIGHT_MIDDLE_FINGER_MCP"]
LEFT_ARM_LANDMARKS_TO_FUSED = ["left elbow", "WRIST", "INDEX_FINGER_MCP", "MIDDLE_FINGER_MCP"]
LEFT_HAND_LANDMARKS_TO_FUSED = ["THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",    
                                "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",    
                                "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",    
                                "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",    
                                "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"]
LANDMARKS_TO_FUSED.extend(LEFT_ARM_LANDMARKS_TO_FUSED)                
LANDMARKS_TO_FUSED.extend(LEFT_HAND_LANDMARKS_TO_FUSED)                
LANDMARKS_TO_FUSED.extend(RIGHT_ARM_LANDMARKS_TO_FUSED)                

for landmark_name in LANDMARKS_TO_FUSED:
    LANDMARKS_TO_FUSED_IDX.append(arm_hand_fused_names.index(landmark_name)) 

In [9]:
target_output = gt_data
time_sleep = .1
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...]  # shape: (144)
    output = output.reshape(3, 48)  # shape: (3, 48)
    output = output.T  # shape: (26, 48)

    lines = body_lines.copy()
    lines.extend(left_arm_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    points_to_plot = np.zeros_like(points)
    points_to_plot[LANDMARKS_TO_FUSED_IDX] = points[LANDMARKS_TO_FUSED_IDX]

    pcd.points = o3d.utility.Vector3dVector(points_to_plot)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points_to_plot)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points_to_plot, axis=0)
    max_x, max_y, max_z = np.max(points_to_plot, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()